### cine21 배우 랭킹 사이트 크롤링
  - 사이트 주소: http://www.cine21.com/rank/person/
  - 요청 방식 확인 방법: 크롬 개발자 모드로 들어가서, Network -> Fetch/XHR -> content 페이지의 요청 방식 확인
    - Request URL: http://www.cine21.com/rank/person/content
    - Request Method: POST
    - Form data
      - section = 'actor'
      - period_start = '2021-07'
      - gender = 'all'
      - page = 1 

In [ ]:
# 필요한 라이브러리 다운로드

# !pip3 install requests
# !pip3 install bs4
# !pip3 install selenium

In [98]:
import re
import requests
from bs4 import BeautifulSoup

In [ ]:
# cine21_url = 'http://www.cine21.com/rank/person/'
# res = requests.get(cine21_url)
# print(res.text) 

# get으로 할 경우 배우 리스트가 응답으로 반환되지 않음(사이트마다 다름)

In [100]:
cine21_url = 'http://www.cine21.com/rank/person/content'
actors_info_list = list()

# 1~10 page 크롤링
for page in range(1,10):

    post_data = dict()
    post_data['section'] = 'actor'
    post_data['period_start'] = '2022-01'
    post_data['gender'] = 'all'
    post_data['page'] = page

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.text, 'html.parser')
    actors = soup.select('li.people_li div.name')

    # 추가한 부분
    actor_score = soup.select('li.people_li ul.num_info li strong')
    actor_movies = soup.select('li.people_li ul.mov_list')
    actor_rank = soup.select('li.people_li span.grade')

    for index, actor in enumerate(actors):
        actor_info_dict = dict()
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_info_dict['이름'] = actor_name

        # 추가한 부분
        actor_info_dict['흥행지수'] = re.sub(',', '', actor_score[index].text)
        actor_movie = actor_movies[index].select('li a span')
        actor_movie = [ re.sub('<span>|</span>', '', x.text) for x in actor_movie ]
        actor_info_dict['출연영화'] = actor_movie
        actor_info_dict['랭킹'] = actor_rank[index].text
        
        # 상세 정보
        actor_link = 'http://www.cine21.com' + actor.select_one('a')['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.text, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')
        
        for actor_detail in actor_details:
            actor_detail_key = actor_detail.select_one('span.tit').text
            actor_detail_value = re.sub('<span.*>.*</span>', '', str(actor_detail))
            actor_detail_value = re.sub('<.*?>', '', actor_detail_value)
            actor_info_dict[actor_detail_key] = actor_detail_value
            
        actors_info_list.append(actor_info_dict)    



actors_info_list

[{'다른 이름': '김하늘',
  '랭킹': '1',
  '생년월일': '1990-02-21',
  '성별': '남',
  '신장/체중': '181cm, 70kg',
  '이름': '강하늘',
  '직업': '배우',
  '출연영화': ['해적: 도깨비 깃발', '해피 뉴 이어'],
  '학교': '중앙대학교 연극학과',
  '홈페이지': '\nhttp://weibo.com/galpos3?is_hot=1\n',
  '흥행지수': '80090'},
 {'랭킹': '2',
  '생년월일': '1987-02-22',
  '성별': '여',
  '신장/체중': '170cm',
  '이름': '한효주',
  '직업': '배우',
  '출연영화': ['해적: 도깨비 깃발'],
  '취미': '영화감상',
  '학교': '동국대학교 연극영화',
  '홈페이지': '\nhttps://www.facebook.com/hhj.official\n',
  '흥행지수': '68142'},
 {'랭킹': '3',
  '생년월일': '1985-07-14',
  '성별': '남',
  '소속사': '킹콩엔터테인먼트',
  '신장/체중': '190cm',
  '이름': '이광수',
  '직업': '배우',
  '출연영화': ['해적: 도깨비 깃발', '해피 뉴 이어'],
  '홈페이지': '\nhttps://twitter.com/masijacoke85\nhttps://www.instagram.com/masijacoke850714/\n',
  '흥행지수': '60206'},
 {'다른 이름': 'Kwon Sang Woo',
  '랭킹': '4',
  '생년월일': '1976-08-05',
  '성별': '남',
  '소속사': '벨액터스 엔터테인먼트',
  '신장/체중': '183cm, 72kg',
  '이름': '권상우',
  '직업': '배우',
  '출연영화': ['해적: 도깨비 깃발'],
  '취미': '수영, 헬스, 복싱',
  '특기': '농구',
  '학교': '한남대학교 미술교